In [1]:
import pandas as pd
import pdfplumber
import re
import os

In [4]:
# 1) Paths
PDF_PATH   = r"C:\Users\techpc\Downloads\data_entry\Data\sources\fastener_catalog.pdf"
OUTPUT_XLS = r"C:\Users\techpc\Downloads\data_entry\Data\Output\fastener_data_cleaned.xlsx"
os.makedirs(os.path.dirname(OUTPUT_XLS), exist_ok=True)

# 2) Extract table via pdfplumber's table extractor
with pdfplumber.open(PDF_PATH) as pdf:
    page  = pdf.pages[0]
    table = page.extract_table()

# 3) Build DataFrame
# table[0] is the header row, table[1:] the data rows
df = pd.DataFrame(table[1:], columns=table[0])
print("Parsed rows:", len(df))
print(df.head())

# 4) Clean up whitespace & non-ASCII
import re
def clean_txt(s):
    s = re.sub(r'[^\x00-\x7F]', '', str(s))
    s = re.sub(r'[\r\n\t]', ' ', s)
    s = re.sub(r'\s{2,}', ' ', s)
    return s.strip()
for col in df.columns:
    df[col] = df[col].apply(clean_txt)

# 5) Flag duplicates
df['Duplicate'] = df.duplicated(subset=['Item#'], keep=False)

# 6) Save to Excel
df.to_excel(OUTPUT_XLS, index=False)
print("✅ Saved", OUTPUT_XLS)


Parsed rows: 20
    Item#   Brand  Size   Material                     Description   ImageFile
0  FN0001  Brand2   2mm      Brass       Hex head bolt, 2mm, Brass  FN0001.jpg
1  FN0002  Brand3   4mm  Stainless   Hex head bolt, 4mm, Stainless  FN0002.jpg
2  FN0003  Brand4   6mm      Steel       Hex head bolt, 6mm, Steel  FN0003.jpg
3  FN0004  Brand5   8mm      Brass       Hex head bolt, 8mm, Brass  FN0004.jpg
4  FN0005  Brand1  10mm  Stainless  Hex head bolt, 10mm, Stainless  FN0005.jpg
✅ Saved C:\Users\techpc\Downloads\data_entry\Data\Output\fastener_data_cleaned.xlsx
